# Evaluation with Azure AI Foundry

This notebook demonstrates how to evaluate data using custom evaluators and send the results to [Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio).

### Prerequisites

- An Azure subscription.
- An Azure AI Foundry workspace.
- An Azure AI Foundry project.
- An Azure OpenAI resource.

### Install the required packages

```bash
pip install -r requirements.txt
```

### Create the following environment variables or add them to an `.env` file

```bash
AZURE_OPENAI_ENDPOINT=<your-azure-openai-endpoint>
AZURE_OPENAI_API_KEY=<your-azure-openai-api-key>
AZURE_OPENAI_DEPLOYMENT=<your-azure-openai-deployment>
AZURE_OPENAI_API_VERSION=<your-azure-openai-api-version>
AZURE_SUBSCRIPTION_ID=<your-azure-subscription-id>
AZURE_RESOURCE_GROUP=<your-azure-resource-group>
AZURE_AI_FOUNDRY_PROJECT=<your-azure-azure_foundry_project>
```

### References

- [Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio)
- [Evaluate your Generative AI application locally with the Azure AI Evaluation SDK](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluating-direct-and-indirect-attack-jailbreak-vulnerability)

In [1]:
!pip install -r requirements.txt

## Imports

In [5]:
import json
import pandas as pd

In [6]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.tracing import start_trace

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

# start a trace session, and print a url for user to check trace
start_trace()

## Setup Credentials and Configuration

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()


azure_ai_project = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.getenv("AZURE_RESOURCE_GROUP"),
    "project_name": os.getenv("AZURE_AI_FOUNDRY_PROJECT"),
}


model_config = {
    "api_key":os.getenv("AZURE_OPENAI_API_KEY"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.getenv("AZURE_OPENAI_DEPLOYMENT"),
}



configuration = AzureOpenAIModelConfiguration(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

credential = DefaultAzureCredential()

In [8]:
print(azure_ai_project)



{'subscription_id': '0debeb64-562c-44d8-9966-110450d5f9ed', 'resource_group_name': 'paramount-data-analytics', 'project_name': 'paramountda01'}


## Groundedness Evaluator

In [12]:
from azure.ai.evaluation import GroundednessProEvaluator, GroundednessEvaluator

# Initialazing Groundedness and Groundedness Pro evaluators
groundedness_eval = GroundednessEvaluator(model_config, threshold=3)

query_response_pass = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions \"ordinarily incident to travel.\" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption. In other words, because cargo freight shipments (or \"parts\" thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as \"accompanied baggage\" or otherwise incident to travel for personal use. The commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services."
)

query_response_fail = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel. The guidance in [Case No. IA-16194] also emphasizes that in order to fully determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include any components (for example, ancillary goods or services) that may fall outside the direct facilitation of travel. Moreover, it is noted that the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR) (see commentary in [Case No. IA-16194]), so if any counterparties are so designated, the exemption would not apply. In summary, for U.S. airlines: • If an interline ticketing and baggage or special prorate agreement is strictly limited to activities directly incident to international travel (e.g., issuing tickets, coordinating baggage transfer, or proration of travel fares), then such transactions are exempt under the travel exemption. • If the agreements include additional non–travel-related components or involve counterparties designated under certain sanctions (such as under the GTSR), then the exemption may not apply. This analysis is based on the information and guidance provided in [Case No. IA-16194]."
)

# Running Groundedness Evaluator on a query and response pair
groundedness_score = groundedness_eval(
    **query_response_fail
)
print(groundedness_score)



{'groundedness': 2.0, 'gpt_groundedness': 2.0, 'groundedness_reason': 'The response is related to the topic of travel exemptions but does not directly address the specific query about apartments for cargo freight. It provides detailed information about interline ticketing and prorate agreements, which is not relevant to the query.', 'groundedness_result': 'fail', 'groundedness_threshold': 3}


In [23]:
records = []
with open("data/data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        result = groundedness_eval(
            query=item["query"],
            context=item.get("context", ""),
            response=item["ground_truth"]
        )
        records.append({
            "query": item["query"],
            "response": item["ground_truth"],
            "groundedness": result["groundedness"],
            "groundedness_reason": result["groundedness_reason"],
            "groundedness_result": result["groundedness_result"],
            "groundedness_threshold": result["groundedness_threshold"],
        })

# Load into a DataFrame and display
df = pd.DataFrame(records)
pd.set_option('display.max_colwidth', None)
display(df)


,query,response,groundedness,groundedness_reason,groundedness_result,groundedness_threshold
0,are partments for cargo freight exempt under the travel exemption?,"No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions ""ordinarily incident to travel."" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption.\nIn other words, because cargo freight shipments (or “parts” thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as “accompanied baggage” or otherwise incident to travel for personal use.\nThe commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services.",5.0,"The response is fully correct and complete, providing a detailed explanation based on specific regulations and directly addressing the query.",pass,3
1,are Interline Ticketing and Baggage and Special Prorate Agreements exempt under the travel exemption?,"Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel.\nThe guidance in [Case No. IA-16194] also emphasizes that to determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include components that fall outside the direct facilitation of travel. Moreover, the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR).\nSummary:\n• If the agreement is strictly limited to activities directly incident to travel (ticketing, baggage transfer, fare proration), it is exempt.\n• If it includes non-travel components or involves designated counterparties, it is not.\nThis analysis is based on [Case No. IA-16194].",5.0,"The RESPONSE is fully correct and complete, directly addressing the QUERY with precise information from the CONTEXT.",pass,3
2,"Is paying ""landings fees and ground handling payments"" to a blocked airport operator in connection with a charter flight exempt under the travel exemption?","No. According to [Case No. Ukraine-EO13662-2018-353695-1] and the related discussion in “Notable Examples Illustrating the Scope of the IEEPA Travel Exemption,” payments for landing fees and ground-handling services to a blocked airport operator are not treated as transactions “ordinarily incident to travel” and therefore are not covered by the travel exemption.\nThe case notes that although certain charter-flight operations may be authorized if they lack a U.S. nexus, OFAC did not indicate that the travel exemption applies to fees paid to blocked operators. The cited examples explicitly list su

## Relevance Evaluator

In [13]:
from azure.ai.evaluation import RelevanceEvaluator

# Initialazing Groundedness and Groundedness Pro evaluators
relevance_eval = RelevanceEvaluator(model_config, threshold=3)

query_response = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions \"ordinarily incident to travel.\" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption.In other words, because cargo freight shipments (or “parts” thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as “accompanied baggage” or otherwise incident to travel for personal use.The commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services.."
)

query_response_fail = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel. The guidance in [Case No. IA-16194] also emphasizes that in order to fully determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include any components (for example, ancillary goods or services) that may fall outside the direct facilitation of travel. Moreover, it is noted that the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR) (see commentary in [Case No. IA-16194]), so if any counterparties are so designated, the exemption would not apply. In summary, for U.S. airlines: • If an interline ticketing and baggage or special prorate agreement is strictly limited to activities directly incident to international travel (e.g., issuing tickets, coordinating baggage transfer, or proration of travel fares), then such transactions are exempt under the travel exemption. • If the agreements include additional non–travel-related components or involve counterparties designated under certain sanctions (such as under the GTSR), then the exemption may not apply. This analysis is based on the information and guidance provided in [Case No. IA-16194]."
)


# Running Groundedness Evaluator on a query and response pair
relevance_eval_score = relevance_eval(
    **query_response_pass
)
print(relevance_eval_score)

{'relevance': 4.0, 'gpt_relevance': 4.0, 'relevance_reason': 'The RESPONSE fully addresses the QUERY with accurate and complete information, including references to specific regulations and a detailed explanation of why cargo freight is not exempt under the travel exemption.', 'relevance_result': 'pass', 'relevance_threshold': 3}


In [22]:
records = []
with open("data/data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        result = relevance_eval(
            query=item["query"],
            context=item.get("context", ""),
            response=item["ground_truth"]
        )
        records.append({
            "query":       item["query"],
            "response":    item["ground_truth"],
            "relevance":   result["relevance"],
            "relevant_reason": result["relevance_reason"],
            "relevant_result": result["relevance_result"],
            "relevant_threshold": result["relevance_threshold"],
         
        })

# 3. Load into a DataFrame and display
df = pd.DataFrame(records)
pd.set_option('display.max_colwidth', None)
display(df)

,query,response,relevance,relevant_reason,relevant_result,relevant_threshold
0,are partments for cargo freight exempt under the travel exemption?,"No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions ""ordinarily incident to travel."" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption.\nIn other words, because cargo freight shipments (or “parts” thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as “accompanied baggage” or otherwise incident to travel for personal use.\nThe commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services.",4.0,"The RESPONSE fully addresses the QUERY with accurate and complete information, explaining why cargo freight compartments are not exempt under the travel exemption. It includes all essential details required for a comprehensive understanding.",pass,3
1,are Interline Ticketing and Baggage and Special Prorate Agreements exempt under the travel exemption?,"Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel.\nThe guidance in [Case No. IA-16194] also emphasizes that to determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include components that fall outside the direct facilitation of travel. Moreover, the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR).\nSummary:\n• If the agreement is strictly limited to activities directly incident to travel (ticketing, baggage transfer, fare proration), it is exempt.\n• If it includes non-travel components or involves designated counterparties, it is not.\nThis analysis is based on [Case No. IA-16194].",5.0,"The RESPONSE fully addresses the QUERY with accurate and complete information, including relevant insights and conditions for the exemption.",pass,3
2,"Is paying ""landings fees and ground handling payments"" to a blocked airport operator in connection with a charter flight exempt under the travel exemption?","No. According to [Case No. Ukraine-EO13662-2018-353695-1] and the related discussion in “Notable Examples Illustrating the Scope of the IEEPA Travel Exemption,” payments for landing fees and ground-handling services to a blocked airport operator are not treated as transactions “ordinarily incident to travel” and therefore are not covered by the travel exemption.\nThe case notes that although certain charter-flight operations may be authorized if they lack a U.S. nexus, OFAC did not indica

## Friendliness Evaluator

In [ ]:
from friendliness.friendliness import FriendlinessEvaluator

friendliness_eval = FriendlinessEvaluator(configuration)

friendliness_score = friendliness_eval(response="I will not apologize for my behavior!")
print(friendliness_score)

{'score': 1, 'reason': 'The response is defensive and lacks warmth or approachability.'}


## Evaluate with both built-in and custom evaluators

In [9]:
import pathlib
from azure.ai.evaluation import GroundednessProEvaluator, GroundednessEvaluator


from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)
from model_endpoint import ModelEndpoint


content_safety_evaluator = ContentSafetyEvaluator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"

print(path)

results = evaluate(
    evaluation_name="Eval-Run-" + "-" + model_config["azure_deployment"].title(),
    data= "./data/data.jsonl",
    target=ModelEndpoint(model_config),

    evaluators={
        "groundedness": groundedness_eval,
        "content_safety": content_safety_evaluator,
        "coherence": coherence_evaluator,
        "relevance": relevance_evaluator,
        "groundedness": groundedness_evaluator,
        "fluency": fluency_evaluator,
        "similarity": similarity_evaluator,
        "friendliness": friendliness_eval #custom evaluator
    },
    # column mapping
    evaluator_config={
        "content_safety": {"column_mapping": {"query": "${data.query}", "response": "${target.response}"}},
        "coherence": {"column_mapping": {"response": "${target.response}", "query": "${data.query}"}},
        "relevance": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "groundedness": {
            "column_mapping": {
                "response": "${target.response}",
                "context": "${data.context}",
                "query": "${data.query}",
            }
        },
        "fluency": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "similarity": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "friendliness": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"
            }
        }
    },
    # Optionally provide your Azure AI project information to track your evaluation results in your Azure AI project
    azure_ai_project = azure_ai_project,
    # # Optionally provide an output path to dump a json of metric summary, row level data and metric and Azure AI project URL
    output_path="./results.jsonl"

)



c:\Users\akaruparti\OneDrive - Microsoft\Documents\Code With\Akin Gump\evals/data.jsonl
{'api_key': 'e058bda1c87b4de297a562abe6439ccb', 'azure_endpoint': 'https://paramountda2801552698.openai.azure.com/', 'azure_deployment': 'gpt-4o', 'type': 'azure_openai', 'api_version': '2024-02-15-preview'}


NameError: name 'groundedness_eval' is not defined

In [ ]:
import os
import pathlib
from pathlib import Path

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)
from model_endpoint import ModelEndpoint

# Disable local snapshot to speed up evaluation
os.environ["PROMPTFLOW_DISABLE_LOCAL_SNAPSHOT"] = "true"

content_safety_evaluator = ContentSafetyEvaluator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

# Create proper output path
output_path = str(Path.cwd() / "results.jsonl")
print(f"Output path: {output_path}")

# Pre-create the results file to avoid snapshot copy issues
Path("results.jsonl").touch()

results = evaluate(
    evaluation_name="Eval-Run-" + "-" + model_config["azure_deployment"].title(),
    data= "./data/data_2.jsonl",
    target=ModelEndpoint(model_config),
    evaluators={
        "content_safety": content_safety_evaluator,
        "coherence": coherence_evaluator,
        "relevance": relevance_evaluator,
        "groundedness": groundedness_evaluator,
        "fluency": fluency_evaluator,
        "similarity": similarity_evaluator,
        #"friendliness": friendliness_eval #custom evaluator
    },
    # column mapping
    evaluator_config={
        "content_safety": {"column_mapping": {"query": "${data.query}", "response": "${data.response}"}},
        "coherence": {"column_mapping": {"response": "${data.response}", "query": "${data.query}"}},
        "relevance": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "groundedness": {
            "column_mapping": {
                "response": "${data.response}",
                "context": "${data.context}",
                "query": "${data.query}",
            }
        },
        "fluency": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "similarity": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "friendliness": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"
            }
        }
    },
    # Optionally provide your Azure AI project information to track your evaluation results in your Azure AI project
    azure_ai_project = azure_ai_project,
    # Use proper output path
    output_path=output_path
)



[2025-06-16 16:24:21 -0400][promptflow][WARNING] - Found existing c:/Users/akaruparti/OneDrive - Microsoft/Documents/Code With/Akin Gump/evals/flow.flex.yaml, will not respect it in runtime.


c:\Users\akaruparti\OneDrive - Microsoft\Documents\Code With\Akin Gump\evals/data__2.jsonl
{'api_key': 'e058bda1c87b4de297a562abe6439ccb', 'azure_endpoint': 'https://paramountda2801552698.openai.azure.com/', 'azure_deployment': 'gpt-4o', 'type': 'azure_openai', 'api_version': '2024-02-15-preview'}


[2025-06-16 16:25:23 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta timeout after 60 seconds, terminate the process.
[2025-06-16 16:25:23 -0400][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_None_20250616_162421_520056, log path: C:\Users\akaruparti\.promptflow\.runs\azure_ai_evaluation_evaluators_None_20250616_162421_520056\logs.txt


EvaluationException: (InternalError) [('C:/Users/akaruparti/OneDrive - Microsoft/Documents/Code With/Akin Gump/evals\\results.jsonl', 'C:\\Users\\akaruparti\\.promptflow\\.runs\\azure_ai_evaluation_evaluators_None_20250616_162421_520056\\snapshot\\results.jsonl', "[Errno 2] No such file or directory: 'C:/Users/akaruparti/OneDrive - Microsoft/Documents/Code With/Akin Gump/evals\\\\results.jsonl'")]

In [2]:
import json
import pandas as pd
from IPython.display import display

# Load the JSON content from the file "results.jsonl"
with open("results.jsonl", "r") as f:
	jsonl_text = f.read()

data = json.loads(jsonl_text)

# Create a DataFrame using the "rows" key from the JSON data
df = pd.DataFrame(data.get("rows", []))

# Convert the DataFrame to CSV format as a string
csv_content = df.to_csv(index=False)
# Display the DataFrame as a table in the notebook
display(df)


,outputs.query,outputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.groundedness.groundedness,outputs.groundedness.gpt_groundedness,outputs.groundedness.groundedness_reason,outputs.groundedness.groundedness_result,outputs.groundedness.groundedness_threshold,...,outputs.relevance.relevance_result,outputs.relevance.relevance_threshold,outputs.fluency.fluency,outputs.fluency.gpt_fluency,outputs.fluency.fluency_reason,outputs.fluency.fluency_result,outputs.fluency.fluency_threshold,outputs.friendliness.score,outputs.friendliness.reason,line_number
0,What is the capital of France?,The capital of France is Paris.,What is the capital of France?,France is the country in Europe.,Paris,3,3,The RESPONSE introduces information about the ...,pass,3,...,pass,3,3,3,The response is clear and grammatically correc...,pass,3,3,"The response is neutral, providing factual inf...",0
1,Which tent is the most waterproof?,"When looking for the most waterproof tent, the...",Which tent is the most waterproof?,"#TrailMaster X4 Tent, price $250,## BrandOutdo...",The TrailMaster X4 tent has a rainfly waterpro...,1,1,The RESPONSE is completely ungrounded as it in...,fail,3,...,pass,3,4,4,The response is well-articulated with good con...,pass,3,4,"The response is mostly friendly, providing det...",1
2,Which camping table is the lightest?,The lightest camping table available can vary ...,Which camping table is the lightest?,"#BaseCamp Folding Table, price $60,## BrandCam...",The BaseCamp Folding Table has a weight of 15 lbs,1,1,The RESPONSE is entirely unrelated to the CONT...,fail,3,...,pass,3,4,4,"The response is well-articulated, with good co...",pass,3,4,"The response is informative and helpful, provi...",2
3,How much does TrailWalker Hiking Shoes cost?,I don't have real-time access to current prici...,How much does TrailWalker Hiking Shoes cost?,"#TrailWalker Hiking Shoes, price $110## BrandT...",The TrailWalker Hiking Shoes are priced at $110,3,3,The RESPONSE does not include the specific det...,pass,3,...,pass,3,4,4,"The response is well-articulated, demonstratin...",pass,3,4,"The response is informative and helpful, provi...",3


In [18]:
df.to_csv("results.csv", index=False)